# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#loading the data file 
data_file = "Output/fileOne.csv"
city_data = pd.read_csv(data_file)


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Store latitude and longitude in locations
locations = city_data[["Lat", "Lng"]]

# Fill NaN values and convert to float
Humidity = city_data["Humidity"].astype(float)


In [7]:
fig = gmaps.figure(map_type="HYBRID")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=300,
                                 point_radius=5)
#plot the figure on gmaps
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
#dropping rows based on tepmerature and storing data in vacation dataframe
vacation_df = city_data.loc[(city_data["Max Temp"] <= 300) & (city_data["Max Temp"] >= 295)
                            & (city_data["Cloudiness"] <= 50), :]


In [9]:
#dropping null values
vacation_df =vacation_df.dropna()
vacation_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
4,Hambantota,20,LK,1584232885,100,6.12,81.12,296.15,2.60
8,Bambous Virieux,40,MU,1584232684,88,-20.34,57.76,298.15,1.00
11,Rikitea,0,PF,1584232677,78,-23.12,-134.97,298.85,5.42
12,Ixtapa,5,MX,1584232972,74,20.70,-105.20,299.15,4.60
28,Souillac,40,MU,1584232931,88,-20.52,57.52,298.15,1.00
31,Filingué,0,NE,1584233265,13,14.35,3.32,297.73,4.68
56,Arraial do Cabo,40,BR,1584232775,94,-22.97,-42.03,297.15,1.50
59,Tessalit,49,ML,1584232655,12,20.20,1.01,296.87,5.30
63,Bambanglipuro,3,ID,1584233268,85,-7.95,110.28,298.86,1.27
90,Cabo San Lucas,40,MX,1584233271,50,22.89,-109.91,297.15,5.10


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = vacation_df
#adding column for hotel name 
hotel_df["Hotel Name"] = ""

In [11]:
#using iterrows to iterate through hotels_df dataframe
for index, row in hotel_df.iterrows():
    
    rowlat = row["Lat"]
    rowlng = row["Lng"]
    location = str(rowlat)+","+ str(rowlng)
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {"location": {location},"radius":5000 ,"type": "hotel","key": g_key}
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {location}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
            print(f"Closest hotel is {results[0]['name']}.")
            #storing results in Hotel column in df
            hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        print("------------")


Retrieving Results for Index 4: 6.12,81.12.
Missing field/result... skipping.
------------
Retrieving Results for Index 8: -20.34,57.76.
Closest hotel is Vieux Grand Port.
Retrieving Results for Index 11: -23.12,-134.97.
Closest hotel is Rikitea.
Retrieving Results for Index 12: 20.7,-105.2.
Closest hotel is Puerto Vallarta.
Retrieving Results for Index 28: -20.52,57.52.
Closest hotel is Souillac.
Retrieving Results for Index 31: 14.35,3.32.
Closest hotel is Filingué.
Retrieving Results for Index 56: -22.97,-42.03.
Closest hotel is Cabo Frio.
Retrieving Results for Index 59: 20.2,1.01.
Closest hotel is Achemelmel.
Retrieving Results for Index 63: -7.95,110.28.
Closest hotel is Bantul.
Retrieving Results for Index 90: 22.89,-109.91.
Closest hotel is Cabo San Lucas.
Retrieving Results for Index 91: -33.7,-53.46.
Missing field/result... skipping.
------------
Retrieving Results for Index 96: 33.19,-82.53.
Closest hotel is Avera.
Retrieving Results for Index 105: -5.2,-35.46.
Closest hotel

In [12]:
#printing df with hotels
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
4,Hambantota,20,LK,1584232885,100,6.12,81.12,296.15,2.60,
8,Bambous Virieux,40,MU,1584232684,88,-20.34,57.76,298.15,1.00,Vieux Grand Port
11,Rikitea,0,PF,1584232677,78,-23.12,-134.97,298.85,5.42,Rikitea
12,Ixtapa,5,MX,1584232972,74,20.70,-105.20,299.15,4.60,Puerto Vallarta
28,Souillac,40,MU,1584232931,88,-20.52,57.52,298.15,1.00,Souillac
31,Filingué,0,NE,1584233265,13,14.35,3.32,297.73,4.68,Filingué
56,Arraial do Cabo,40,BR,1584232775,94,-22.97,-42.03,297.15,1.50,Cabo Frio
59,Tessalit,49,ML,1584232655,12,20.20,1.01,296.87,5.30,Achemelmel
63,Bambanglipuro,3,ID,1584233268,85,-7.95,110.28,298.86,1.27,Bantul
90,Cabo San Lucas,40,MX,1584233271,50,22.89,-109.91,297.15,5.10,Cabo San Lucas


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]



In [15]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))